In [1]:
import pandas as pd
import sqlite3

conn=sqlite3.connect('../database/sql_solution.db')

In [2]:
employees_df = pd.read_csv("../data/employees.csv", parse_dates=["join_date", "resign_date"])

employees_df.to_sql(name="employees", con=conn, if_exists='replace', index=False)
conn.commit()


timesheets_df = pd.read_csv("../data/timesheets.csv", parse_dates=["date"])

timesheets_df.to_sql(name="timesheets", con=conn, if_exists='replace', index=False)
conn.commit()

In [3]:
%load_ext sql
%sql sqlite:///../database/sql_solution.db

Connecting to 'sqlite:///../database/sql_solution.db'

In [4]:
%%sql

SELECT * FROM employees

Running query in 'sqlite:///../database/sql_solution.db'

employe_id,branch_id,salary,join_date,resign_date
1,3,7500000,2018-08-23 00:00:00,None
7,1,7500000,2017-04-28 00:00:00,None
8,1,13000000,2017-04-28 00:00:00,None
9,1,13500000,2017-12-22 00:00:00,2020-10-14 00:00:00
10,1,9000000,2018-03-01 00:00:00,2020-12-20 00:00:00
11,1,8500000,2018-07-02 00:00:00,2019-11-04 00:00:00
12,1,7500000,2018-07-06 00:00:00,2019-10-30 00:00:00
13,1,9500000,2018-07-30 00:00:00,None
14,1,6000000,2018-07-30 00:00:00,None
15,1,6000000,2020-07-01 00:00:00,None


In [5]:
%%sql

SELECT * FROM timesheets

Running query in 'sqlite:///../database/sql_solution.db'

timesheet_id,employee_id,date,checkin,checkout
23907432,66,2019-08-21 00:00:00,08:13:31,17:05:02
23907433,22,2019-08-21 00:00:00,08:56:34,18:00:48
23907434,21,2019-08-21 00:00:00,09:45:08,18:24:06
23907435,63,2019-08-21 00:00:00,09:55:47,None
23907437,60,2019-08-21 00:00:00,09:56:05,17:31:08
23907443,66,2019-08-22 00:00:00,08:28:27,17:20:25
23907444,21,2019-08-22 00:00:00,09:23:37,18:29:51
23907445,60,2019-08-22 00:00:00,None,18:04:33
23907447,22,2019-08-23 00:00:00,08:55:59,17:59:43
23907448,60,2019-08-23 00:00:00,10:34:39,17:25:41


In [8]:
%%sql

CREATE TABLE salary_per_hour (
    year INTEGER,
    month INTEGER,
    branch_id INTEGER,
    salary_per_hour INTEGER
);

Running query in 'sqlite:///../database/sql_solution.db'

++
||
++
++

In [18]:
%%sql


INSERT INTO salary_per_hour (year, month, branch_id, salary_per_hour)
WITH summary AS (
    SELECT 
        strftime('%Y', t.date) AS year,
        strftime('%m', t.date) AS month,
        e.branch_id,
        t.employee_id,
        SUM(e.salary) AS salary_total,
        SUM(
            COALESCE(
                (strftime('%s', t.checkout) - strftime('%s', t.checkin)) / 3600,
                8 # default daily manhour
            )
        ) AS working_hours_total
    FROM
        timesheets t
    LEFT JOIN
        employees e ON t.employee_id = e.employe_id
    WHERE
        t.date IS NOT NULL
    GROUP BY
        year,
        month,
        branch_id
)

SELECT
    year,
    month,
    branch_id,
    ROUND(salary_total / working_hours_total, 2) AS salary_per_hour
FROM
    summary;


Running query in 'sqlite:///../database/sql_solution.db'

239 rows affected.

++
||
++
++

In [19]:
%%sql

SELECT * FROM salary_per_hour

Running query in 'sqlite:///../database/sql_solution.db'

year,month,branch_id,salary_per_hour
2019,8,1,1264705
2019,8,2,1546012
2019,8,3,1625000
2019,9,1,1136424
2019,9,2,1473988
2019,9,3,1466887
2019,9,2590,1357142
2019,9,2623,1388104
2019,9,2625,1678899
2019,9,2626,1296875
